In [162]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import os
import zipfile

In [174]:
### 爬取房地產實價登錄網站 ###

url="https://plvr.land.moi.gov.tw/Login_input?authfailed=true#"

# 配置 ChromeOptions
chrome_options = webdriver.ChromeOptions()

# 在瀏覽器頁面加載時，不會顯示網頁通知。
options.add_argument("--disable-notifications")  

#下載設定
download_path = 'C:\\Users\\gary8\\實價登錄之應用'
prefs = {
    'download.default_directory': download_path,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True,
    'profile.default_content_setting_values.automatic_downloads':1
}
chrome_options.add_experimental_option('prefs', prefs)

chrome = webdriver.Chrome(options=chrome_options)

# 打開網址
chrome.get(url)

# 點擊 : Open data
xpath = '/html/body/div[4]/div/ul/li[4]/a'
OpenData = chrome.find_element("xpath",xpath)
OpenData.click()

main_window_handle = chrome.current_window_handle

# 等待額外頁面打開
WebDriverWait(chrome, 10).until(lambda d: len(d.window_handles) > 1)

#切換新頁面以進行關閉
new_window_handle = [handle for handle in chrome.window_handles if handle != main_window_handle][0]
chrome.switch_to.window(new_window_handle)
chrome.close()

#切換為原頁面
chrome.switch_to.window(main_window_handle)

# 點擊 : 非本期下載
xpath = '/html/body/div[1]/div[2]/div/div/div[2]/ul/li[2]'
NotIssueDownload = chrome.find_element("xpath", xpath)
NotIssueDownload.click()

time.sleep(5)

# 從下拉列表中選擇選項 : CSV檔案格式
xpath = '/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[1]/td[2]/select'
FileFormat = chrome.find_element("xpath", xpath)
Select(FileFormat).select_by_index(2) 

# 勾選選項 : 進階下載、(縣市/交易類別 : 台北新北
xpaths = [
    '/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[3]/td[2]/font/input[2]',
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[7]/td[2]/input",
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[7]/td[3]/input",
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[7]/td[4]/input",
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[8]/td[2]/input",
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[8]/td[3]/input",
    "/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[8]/td[4]/input"
]

for xpath in xpaths:
    Category = chrome.find_element("xpath",xpath)
    Category.click()

In [184]:
# 記錄開始時間
start_time = time.time()

# 0 => 112-3,  => 109-3
for i in range(13,47):
    
    # 從下拉列表中選擇選項 : 發布日期
    xpath = '/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[2]/tbody/tr[2]/td[2]/select'
    SeasonDownload = chrome.find_element("xpath", xpath)
    Select(SeasonDownload).select_by_index(i)   
    
    if (i + 1) % 4 == 4:
        Y = 112 - (i + 1) // 4
        print(f"{Y}S4")
    else :
        Y = 112 - (i + 1) // 4
        S = (i + 1) % 4
        print(f"{Y}S{S}")
        
    # 點擊 : 下載
    xpath = '/html/body/div[1]/div[2]/div/div/div[2]/div[2]/div[2]/div/table[3]/tbody/tr[1]/td[3]'
    Download = chrome.find_element("xpath", xpath)
    Download.click()

    # 等待下載完成（等待10秒，你可以根據實際情況調整等待時間）
    time.sleep(40)

    # 獲取元素的值（value屬性）
    option_value = SeasonDownload.get_attribute("value")

    os.rename('C:/Users/gary8/實價登錄之應用/download.zip', 'C:/Users/gary8/實價登錄之應用/{}.zip'.format(option_value))
    
    time.sleep(10)
    

# 記錄結束時間
end_time = time.time()

# 計算執行時間（以秒為單位）
execution_time = end_time - start_time

print(f"程式執行時間：{execution_time} 秒")

13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
程式執行時間：1730.5041031837463 秒


In [185]:
def real_estate_crawler(year,season):
    # 儲存內容到檔案
    FileName= str(year)+"S"+str(season)+'.zip'

    # 為提取的檔案額外增加資料夾
    folder = str(year)+"S"+str(season)
    if not os.path.isdir(folder):
        os.mkdir(folder)

    # 將檔案解壓縮到資料夾中
    with zipfile.ZipFile(FileName, 'r') as zip_ref:
        zip_ref.extractall(folder)

In [186]:
for year in range(101, 112):
  if year == 112:
    for season in range(1,4):
      print(year, season)
      real_estate_crawler(year, season)
  else:
    for season in range(1,5):
      print(year, season)
      real_estate_crawler(year, season)

101 1
101 2
101 3
101 4
102 1
102 2
102 3
102 4
103 1
103 2
103 3
103 4
104 1
104 2
104 3
104 4
105 1
105 2
105 3
105 4
106 1
106 2
106 3
106 4
107 1
107 2
107 3
107 4
108 1
108 2
108 3
108 4
109 1
109 2
109 3
109 4
110 1
110 2
110 3
110 4
111 1
111 2
111 3
111 4
